In [1]:
import pickle
import numpy as np
import pandas as pd
from collections import defaultdict

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Clean the input files (ADMISSION.csv and DIAGNOSES_ICD.csv)
First of all we have to remove the visits without diagnsis code

In [3]:
path = '/home/user/venvs/old_LORE_env/doctorai/MIMICIII_data/'
output_path = '/home/user/venvs/doctorAI/MIMICIII_data/'
admissionFile = path+'ADMISSIONS.csv'
diagnosisFile = path+'DIAGNOSES_ICD.csv'

The diagnoses file, there are some diagnosis without ICD9 code

In [4]:
diag_csv = pd.read_csv(diagnosisFile)
#there are no duplicates
len(diag_csv.drop_duplicates()), len(diag_csv)

(651047, 651047)

In [5]:
#there are some diagnosis without ICD9 code
ratio_of_null = len(diag_csv[diag_csv.ICD9_CODE.isna()])/float(len(diag_csv))
print('number of admissions without diagnosis code: '+str(len(diag_csv[diag_csv.ICD9_CODE.isna()])))
print('number of patient without diagnosis code: '+str(len(diag_csv[diag_csv.ICD9_CODE.isna()].groupby('SUBJECT_ID'))))
print(str(round(ratio_of_null*100,3))+'% of diagnoses do not have the related ICD9 code associated')
null_diag = diag_csv[diag_csv.ICD9_CODE.isna()]
null_diag.head()

number of admissions without diagnosis code: 47
number of patient without diagnosis code: 47
0.007% of diagnoses do not have the related ICD9 code associated


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
5251,5029,417,102633,NaN,NaN
8354,7969,690,174817,NaN,NaN
9691,10924,937,177274,NaN,NaN
18667,16813,1490,135580,NaN,NaN
27313,37428,3369,126808,NaN,NaN


The admission file, check if the values are missing at random or a particular diagnosis is always missing

In [6]:
admissionFile = path+'ADMISSIONS.csv'
admission_table = pd.read_csv(admissionFile)
admission_without_code = admission_table[(admission_table.SUBJECT_ID.isin(null_diag.SUBJECT_ID.values))&(admission_table.HADM_ID.isin(null_diag.HADM_ID.values))]
admission_without_code.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
425,534,417,102633,2177-03-23 16:17:00,2177-03-23 07:20:00,2177-03-23 07:20:00,URGENT,PHYS REFERRAL/NORMAL DELI,DEAD/EXPIRED,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,NaN,NaN,ORGAN DONOR ACCOUNT,1,1
676,1163,937,177274,2163-01-24 09:29:00,2163-01-26 03:45:00,2163-01-26 03:45:00,EMERGENCY,PHYS REFERRAL/NORMAL DELI,DEAD/EXPIRED,Private,NaN,CATHOLIC,NaN,UNKNOWN/NOT SPECIFIED,NaN,NaN,ORGAN DONOR ACCOUNT,1,1
1121,852,690,174817,2188-06-01 08:03:00,2188-06-01 23:59:00,NaN,EMERGENCY,PHYS REFERRAL/NORMAL DELI,HOME,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,NaN,0,0
2163,1818,1490,135580,2117-12-27 06:48:00,2117-12-27 23:59:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Self Pay,NaN,UNOBTAINABLE,NaN,WHITE,NaN,NaN,NaN,0,1
4237,4060,3369,126808,2111-01-24 11:53:00,2111-01-25 22:40:00,2111-01-25 22:40:00,EMERGENCY,PHYS REFERRAL/NORMAL DELI,DEAD/EXPIRED,Private,NaN,UNOBTAINABLE,SINGLE,WHITE,NaN,NaN,NaN,1,1


In [7]:
admission_without_code.groupby('DIAGNOSIS').size()

DIAGNOSIS
BLADDER CA/SDA                     1
GROSS HEMATURIA,CARDIAC ARREST     1
NON HEALING WOUND/SDA              1
ORGAN DONOR                        8
ORGAN DONOR                        1
ORGAN DONOR ACCOUNT               24
SEPSIS                             1
dtype: int64

In [8]:
admission_without_code.groupby('DISCHARGE_LOCATION').size()

DISCHARGE_LOCATION
DEAD/EXPIRED    30
HOME            17
dtype: int64

In [9]:
print('n. patients with diagnosis "ORGAN DONOR ACCOUNT": '+str(len(admission_table[admission_table.DIAGNOSIS=='ORGAN DONOR ACCOUNT'])))
print('n. patients with diagnosis "ORGAN DONOR ACCOUNT" and missing code: '+str(len(admission_without_code[admission_without_code.DIAGNOSIS=='ORGAN DONOR ACCOUNT'])))

n. patients with diagnosis "ORGAN DONOR ACCOUNT": 35
n. patients with diagnosis "ORGAN DONOR ACCOUNT" and missing code: 24


In [10]:
organ_donor_descr_types = [diag_descr for diag_descr in admission_table.DIAGNOSIS.astype(str).values if 'donor' in diag_descr.lower()]
organ_donor_descr_types

['ORGAN DONOR ACCOUNT',
 'ORGAN DONOR ACCOUNT',
 'ORGAN DONOR ACCOUNT',
 'ORGAN DONOR ACCOUNT',
 'LIVING UNRELATED LIVER DONOR/SDA',
 'ORGAN DONOR ACCOUNT',
 'ORGAN DONOR ACCOUNT',
 'DONOR ACCOUNT',
 'ORGAN DONOR',
 'LIVING RELATED LIVER DONOR/SDA',
 'LIVING ADULT LIVER DONOR/SDA',
 'LIVER DONOR/SDA',
 'ORGAN DONOR ACCOUNT',
 'ORGAN DONOR',
 'ORGAN DONOR',
 'ORGAN DONOR ACCOUNT',
 'ORGAN DONOR ACCOUNT',
 'ORGAN DONOR',
 'ORGAN DONOR ACCOUNT',
 'ORGAN DONOR ACCOUNT',
 'ORGAN DONOR ACCOUNT',
 'ORGAN DONOR ACCOUNT',
 'ORGAN DONOR',
 'ORGAN DONOR',
 'ORGAN DONOR',
 'ORGAN DONOR ACCOUNT',
 'ORGAN DONOR',
 'ORGAN DONOR ACCOUNT',
 'ORGAN DONOR ACCOUNT',
 'ORGAN DONOR ACCOUNT',
 'ORGAN DONOR ACCOUNT',
 'ORGAN DONOR ACCOUNT',
 'LIVING LIVER DONOR/SDA',
 'LIVER DONOR/SDA',
 'ORGAN DONOR ACCOUNT',
 'LIVING ADULT LIVER DONOR/SDA',
 'BILE LEAK S/P LIVING DONOR HEPATECTOMY/SDA',
 'ORGAN DONOR',
 'ORGAN DONOR ACCOUNT',
 'ORGAN DONOR ',
 'ORGAN DONOR ACCOUNT',
 'ORGAN DONOR ACCOUNT',
 'ORGAN DONOR ACC

In [11]:
print('number of patients with organ-donor related diag description: '+str(len(admission_table[admission_table.DIAGNOSIS.isin(organ_donor_descr_types)])))

number of patients with organ-donor related diag description: 54


In [12]:
organ_donors_admissions = admission_table[admission_table.DIAGNOSIS.isin(organ_donor_descr_types)]
pd.merge(organ_donors_admissions,diag_csv,on='HADM_ID').fillna('missing').groupby('ICD9_CODE').size()

ICD9_CODE
0479        1
2639        1
2762        1
2851        2
2869        1
2874        1
2930        1
3051        1
3488        2
4019        3
40390       2
41400       1
4329        1
462         1
486         1
4928        1
5119        1
5180        1
53081       1
5601        1
56781       1
5680        1
57140       1
5715        1
5718        2
57511       1
5768        1
5829        1
587         1
5990        1
7231        1
73300       1
75160       1
7596        1
7907        1
8505        2
99659       1
9974        2
99811       1
99889       1
E8786       1
E9342       1
V596        7
missing    33
dtype: int64

It seems that they are NOT missing at random MOST OF THE MISSING CODES ARE RELATED TO THE ORGAN DONORS, but since it's a low percentage (47 patients), we just ignore it for now.

In [13]:
#we just drop the cases where there is no diagnosis code
len(diag_csv.dropna(subset=['ICD9_CODE'])),len(diag_csv)

(651000, 651047)

In [14]:
#create the clean diagnoses file
clean_diag_csv = diag_csv.dropna(subset=['ICD9_CODE'])
clean_diag_csv.to_csv(path+'clean_DIAGNOSES_ICD.csv',index=False)

In [15]:
#devo togliere anche i ricoveri contenenti una sola visita con codice Nan dal file delle ammissioni
admission_csv = pd.read_csv(admissionFile)
clean_admission_csv = admission_csv[admission_csv.HADM_ID.isin(clean_diag_csv.HADM_ID)]
len(clean_admission_csv),len(admission_csv)
clean_admission_csv.to_csv(path+'clean_ADMISSIONS.csv',index=False)

## Create the input files for doctorAI script from the clean tables

The output of this section will be the output of the preprocessing done by Choi:

* mimic.pids: List of unique Patient IDs. Used for intermediate processing
* mimic.dates: List of List of Python datetime objects. The outer List is for each patient. The inner List is for each visit made by each patient
* mimic.seqs: List of List of List of integer diagnosis codes. The outer List is for each patient. The middle List contains visits made by each patient. The inner List contains the integer diagnosis codes that occurred in each visit
* mimic.types: Python dictionary that maps string diagnosis codes to integer diagnosis codes.

In [16]:
admissionFile = path+'clean_ADMISSIONS.csv'
diagnosisFile = path+'clean_DIAGNOSES_ICD.csv'
outFile = path+'clean_mimic'

In [17]:
clean_diag_csv = pd.read_csv(diagnosisFile)
clean_admission_csv = pd.read_csv(admissionFile)

In [18]:
def convert_to_icd9(dxStr):
    if dxStr.startswith('E'):
        if len(dxStr) > 4: return dxStr[:4] + '.' + dxStr[4:]
        else: return dxStr
    else:
        if len(dxStr) > 3: return dxStr[:3] + '.' + dxStr[3:]
        else: return dxStr

def convert_to_3digit_icd9(dxStr):
    if dxStr.startswith('E'):
        if len(dxStr) > 4: return dxStr[:4]
        else: return dxStr
    else:
        if len(dxStr) > 3: return dxStr[:3]
        else: return dxStr

In [19]:
# This script processes MIMIC-III dataset and builds longitudinal diagnosis records for patients with at least two visits.
# The output data are cPickled, and suitable for training Doctor AI or RETAIN
# Written by Edward Choi (mp2893@gatech.edu)
# Usage: Put this script to the foler where MIMIC-III CSV files are located. Then execute the below command.
# python process_mimic.py ADMISSIONS.csv DIAGNOSES_ICD.csv <output file> 

# Output files
# <output file>.pids: List of unique Patient IDs. Used for intermediate processing
# <output file>.dates: List of List of Python datetime objects. The outer List is for each patient. The inner List is for each visit made by each patient
# <output file>.seqs: List of List of List of integer diagnosis codes. The outer List is for each patient. The middle List contains visits made by each patient. The inner List contains the integer diagnosis codes that occurred in each visit
# <output file>.types: Python dictionary that maps string diagnosis codes to integer diagnosis codes.


import sys
import pickle
from datetime import datetime


if __name__ == '__main__':

    print('Building pid-admission mapping, admission-date mapping')
    pidAdmMap = {}
    admDateMap = {}
    infd = open(admissionFile, 'r')
    infd.readline()
    for line in infd:
        tokens = line.strip().split(',')
        pid = int(tokens[1])
        admId = int(tokens[2])
        admTime = datetime.strptime(tokens[3], '%Y-%m-%d %H:%M:%S')
        admDateMap[admId] = admTime
        if pid in pidAdmMap: pidAdmMap[pid].append(admId)
        else: pidAdmMap[pid] = [admId]
    infd.close()

    print('Building admission-dxList mapping')
    admDxMap = {}
    infd = open(diagnosisFile, 'r')
    infd.readline()
    for line in infd:
        tokens = line.strip().split(',')
        ##########debugging##########################
        #print(tokens)
        #print(tokens[4])
        #############################################

        admId = int(tokens[2])
        #dxStr = 'D_' + convert_to_icd9(tokens[4][1:-1]) ############## Uncomment this line and comment the line below, if you want to use the entire ICD9 digits.
        dxStr = 'D_' + convert_to_icd9(tokens[4])
        #dxStr = 'D_' + convert_to_3digit_icd9(tokens[4][1:-1])
        if admId in admDxMap: admDxMap[admId].append(dxStr)
        else: admDxMap[admId] = [dxStr]
    infd.close()

    print('Building pid-sortedVisits mapping')
    pidSeqMap = {}
    for pid, admIdList in pidAdmMap.items():
        if len(admIdList) < 2: continue
        sortedList = sorted([(admDateMap[admId], admDxMap[admId]) for admId in admIdList])
        pidSeqMap[pid] = sortedList
    
    print('Building pids, dates, strSeqs')
    pids = []
    dates = []
    seqs = []
    for pid, visits in pidSeqMap.items():
        pids.append(pid)
        seq = []
        date = []
        for visit in visits:
            date.append(visit[0])
            seq.append(visit[1])
        dates.append(date)
        seqs.append(seq)

    print('Converting strSeqs to intSeqs, and making types')
    types = {}
    newSeqs = []
    for patient in seqs:
        newPatient = []
        for visit in patient:
            newVisit = []
            for code in visit:
                if code in types:
                    newVisit.append(types[code])
                else:
                    types[code] = len(types)
                    newVisit.append(types[code])
            newPatient.append(newVisit)
        newSeqs.append(newPatient)
        
    pickle.dump(pids, open(outFile+'.pids', 'wb'), -1)
    pickle.dump(dates, open(outFile+'.dates', 'wb'), -1)
    pickle.dump(newSeqs, open(outFile+'.seqs', 'wb'), -1)
    pickle.dump(types, open(outFile+'.types', 'wb'), -1)

Building pid-admission mapping, admission-date mapping
Building admission-dxList mapping
Building pid-sortedVisits mapping
Building pids, dates, strSeqs
Converting strSeqs to intSeqs, and making types


In [20]:
#number of unique codes into seqs
flat_list = []

for patient in newSeqs:
    for visit in patient:
        for code in visit:
            flat_list.append(code)
            
len(set(flat_list))

4880

Doctor AI's training dataset needs to be a Python Pickled list of list of list. 
Each list corresponds to patients, visits, and medical codes (e.g. diagnosis codes, medication codes, procedure codes, etc.) First, medical codes need to be converted to an integer. Then a single visit can be seen as a list of integers. Then a patient can be seen as a list of visits. For example, [5,8,15] means the patient was assigned with code 5, 8, and 15 at a certain visit. If a patient made two visits [1,2,3] and [4,5,6,7], it can be converted to a list of list [[1,2,3], [4,5,6,7]]. Multiple patients can be represented as [[[1,2,3], [4,5,6,7]], [[2,4], [8,3,1], [3]]], which means there are two patients where the first patient made two visits and the second patient made three visits. This list of list of list needs to be pickled using cPickle. We will refer to this file as the **"visit file"**.

### Create the visit file, the label file and their splitting

In [21]:
pids = pickle.load( open( path+"clean_mimic.pids", "rb" ) )
seqs = pickle.load( open( path+"clean_mimic.seqs", "rb" ) )
types = pickle.load( open( path+"clean_mimic.types", "rb" ) )
#dates = pickle.load( open( path+"clean_mimic.dates", "rb" ))

* **"visit file"** is *clean_mimic.seqs*
* **"label file"** could be the same as the visit file, or a grouped version using CCS groupers

we created a dictionary with key the ICD9 code and value the CSS grouper from this table https://www.nber.org/data/icd-ccs-crosswalk.html

In [22]:
#dictionary with key the ICD9 code and value the CSS grouper
css_grouper = pickle.load( open( "./conv_dict", "rb" ) )
css_grouper

{0: [0],
 '0010': [135],
 '0011': [135],
 '0019': [135],
 '0020': [135],
 '0021': [135],
 '0022': [135],
 '0023': [135],
 '0029': [135],
 '0030': [135],
 '0031': [2],
 '00320': [135],
 '00321': [76],
 '00322': [122],
 '00323': [201],
 '00324': [201],
 '00329': [135],
 '0038': [135],
 '0039': [135],
 '0040': [135],
 '0041': [135],
 '0042': [135],
 '0043': [135],
 '0048': [135],
 '0049': [135],
 '0050': [135],
 '0051': [135],
 '0052': [135],
 '0053': [135],
 '0054': [135],
 '0058': [135],
 '00581': [135],
 '00589': [135],
 '0059': [135],
 '0060': [135],
 '0061': [135],
 '0062': [135],
 '0063': [135],
 '0064': [135],
 '0065': [135],
 '0066': [135],
 '0068': [135],
 '0069': [135],
 '0070': [135],
 '0071': [135],
 '0072': [135],
 '0073': [135],
 '0074': [135],
 '0075': [135],
 '0078': [135],
 '0079': [135],
 '0080': [135],
 '00800': [135],
 '00801': [135],
 '00802': [135],
 '00803': [135],
 '00804': [135],
 '00809': [135],
 '0081': [135],
 '0082': [135],
 '0083': [135],
 '00841': [135],
 '0

In [23]:
#convert ICD9 codes with format:
#    D_414.01
#into this readable format:
#      41401
#
#creates a dictionary with key the clean ICD9 code and value the integer of the preprocessing

new_dict_types = defaultdict(int)

for item in types.items():
    clean_ICD9 = item[0][2:].replace('.','')
    if len(clean_ICD9)==0:
        #tengo traccia dello strano ICD9 
        clean_ICD9 = 'unknown'
    new_dict_types[clean_ICD9] = item[1]

In [24]:
# maps each integer value of the preprocessing to its grouper
# dictionary with key the integer of the preprocessing and value its ccs grouper
mimic_int_to_ccs_dict = defaultdict(str)

for item in new_dict_types.items():
    associated_int = int(item[1])
    ICD9_code = item[0]
    if ICD9_code=='unknown':
        mimic_int_to_ccs_dict[associated_int] = 'unknown'
    else:    
        mimic_int_to_ccs_dict[associated_int] = css_grouper[ICD9_code][0]

In [25]:
new_ccs_seqs = []

for patient in seqs:
    new_ccs_patient = []
    for visit in patient:
        new_ccs_visit = [mimic_int_to_ccs_dict[code] for code in visit]
        new_ccs_patient.append(new_ccs_visit)
        #print(visit)
        #print(new_ccs_visit)
        #print('end visit')
    new_ccs_seqs.append(new_ccs_patient)
    #print('end patient')
    #print()
    #print()

In [26]:
css_codes = []
for patient in seqs:
    for visit in patient:
        for code in visit:
            css_codes.append(mimic_int_to_ccs_dict[code])
len(set(css_codes))

272

Va fatto un mapping anche tra i codici CCS e gli interi perchè internamente doctorAI controlla che il numero che gli dai come numero massimo di codici unici non sia superato da nessuno dei codici che gli hai dato, per capirci, altrimenti vedi questo errore: 

In [27]:
print('Converting strSeqs to intSeqs, and making CCStypes')
CCStypes = {}
newCSSSeqs = []
for patient in new_ccs_seqs:
    newPatient = []
    for visit in patient:
        newVisit = []
        for code in visit:
            if code in CCStypes:
                newVisit.append(CCStypes[code])
            else:
                CCStypes[code] = len(CCStypes)
                newVisit.append(CCStypes[code])
        newPatient.append(newVisit)
    newCSSSeqs.append(newPatient)

Converting strSeqs to intSeqs, and making CCStypes


In [28]:
#newCSSSeqs

In [29]:
visit_file = seqs
label_file = newCSSSeqs

In [30]:
pickle.dump(visit_file, open(output_path+'visit_complete', 'wb'), protocol=-1)
pickle.dump(label_file, open(output_path+'label_complete', 'wb'), protocol=-1)

In [31]:
len(visit_file),len(label_file)

(7499, 7499)

The "visit file" and "label file" need to have 3 sets respectively: training set, validation set, and test set. The file extension must be ".train", ".valid", and ".test" respectivley.
For example, if you want to use a file named "my_visit_sequences" as the "visit file", then Doctor AI will try to load "my_visit_sequences.train", "my_visit_sequences.valid", and "my_visit_sequences.test".
This is also true for the "label file"


**NB: il pickle va dumpato con protocol=2 perchè altrimenti python2 non capisce**

In [ ]:
########## SENZA SHUFFLE
visit_train = visit_file[:5000]
visit_valid = visit_file[5000:6250]
visit_test = visit_file[6250:]

label_train = label_file[:5000]
label_valid = label_file[5000:6250]
label_test = label_file[6250:]


pickle.dump(visit_train, open(output_path+'visit.train', 'wb'), protocol=-1)
pickle.dump(visit_valid, open(output_path+'visit.valid', 'wb'), protocol=-1)
pickle.dump(visit_test, open(output_path+'visit.test', 'wb'), protocol=-1)

pickle.dump(label_train, open(output_path+'label.train', 'wb'), protocol=-1)
pickle.dump(label_valid, open(output_path+'label.valid', 'wb'), protocol=-1)
pickle.dump(label_test, open(output_path+'label.test', 'wb'), protocol=-1)

DATA SHUFFLE:

In [33]:
########### CON SHUFFLE
from sklearn.model_selection import train_test_split

visit_train, visit_Test, label_train, label_Test = train_test_split(visit_file, label_file, test_size=0.33, random_state=42)
visit_test, visit_valid, label_test, label_valid = train_test_split(visit_Test, label_Test, test_size=0.33, random_state=42)



pickle.dump(visit_train, open(output_path+'visit.train', 'wb'), protocol=-1)
pickle.dump(visit_valid, open(output_path+'visit.valid', 'wb'), protocol=-1)
pickle.dump(visit_test, open(output_path+'visit.test', 'wb'), protocol=-1)

pickle.dump(label_train, open(output_path+'label.train', 'wb'), protocol=-1)
pickle.dump(label_valid, open(output_path+'label.valid', 'wb'), protocol=-1)
pickle.dump(label_test, open(output_path+'label.test', 'wb'), protocol=-1)

len(visit_train),len(visit_test),len(visit_valid)

(5024, 1658, 817)